# Deep RL hands-on by Maxim Lapan
## Chapter 4 The Corss-Entropy Method

* conda activate gym 
 
### Method step

1. Play N number of episodes using our current model and environment.
1. Calculate the total reward for every episode and decide on a reward boundary. Usually, we use some percentile of all rewards, such as 50th or 70th.
1. Throw away all episodes with a reward below the boundary.
1. Train on the remaining "elite" episodes using obervations as the input and issued actions as the desired outtput.
1. Repeat from step 1 until we become satisfied with the result.

## Cart pole

In [116]:
import gym
from collections import namedtuple
import numpy as np
import tensorflow as tf

import torch
import torch.nn as nn
import torch.optim as optim
import datetime

In [117]:
# set hyperparameters
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [118]:
# set basic neural net module with FC layer and ReLU
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


In [119]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [120]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=0) # below the input obs has no [], I use dim=0 instead of dim=1 in the original code
    while True:
        obs_v = torch.FloatTensor(obs) # original code use [obs], but I deleted it. 
        # If you use [obs], the output data will be also embraced with [], like [[0.9, 0.1]]
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy() # I deleted [0], there is no additional dimension in the output.
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [121]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue # forward to next iter.
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [130]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="recording1_cartpole_cross", force=True) # if you wanna record the video
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)

In [131]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convient to record the date with current time
LOGDIR = './tmp/cross_entroy/cartpole/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR) # tf 2.0 has changed the modules of tensorboard
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v.item(), iter_no)
        tf.summary.scalar("reward_bound", reward_b, iter_no)
        tf.summary.scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break

0: loss=0.671, reward_mean=23.9, reward_bound=29.5
1: loss=0.676, reward_mean=21.4, reward_bound=25.5
2: loss=0.661, reward_mean=24.4, reward_bound=23.5
3: loss=0.657, reward_mean=31.8, reward_bound=38.5
4: loss=0.625, reward_mean=65.4, reward_bound=79.0
5: loss=0.618, reward_mean=39.1, reward_bound=44.5
6: loss=0.601, reward_mean=63.2, reward_bound=73.0
7: loss=0.602, reward_mean=53.9, reward_bound=64.5
8: loss=0.593, reward_mean=50.6, reward_bound=58.5
9: loss=0.582, reward_mean=68.9, reward_bound=75.0
10: loss=0.570, reward_mean=63.0, reward_bound=72.0
11: loss=0.570, reward_mean=70.9, reward_bound=83.0
12: loss=0.567, reward_mean=74.7, reward_bound=88.0
13: loss=0.543, reward_mean=80.4, reward_bound=85.0
14: loss=0.557, reward_mean=71.8, reward_bound=78.0
15: loss=0.546, reward_mean=86.7, reward_bound=95.0
16: loss=0.535, reward_mean=93.6, reward_bound=116.0
17: loss=0.528, reward_mean=91.1, reward_bound=101.5
18: loss=0.549, reward_mean=97.2, reward_bound=104.0
19: loss=0.555, rew

## Frozen Lake

In [124]:
e = gym.make("FrozenLake-v0")
print(e.observation_space)
print(e.action_space)
e.reset()
e.render()

Discrete(16)
Discrete(4)

SFFF
FHFH
FFFH
HFFG


In [125]:
# use same hyperparameter, Net module, Episode tuple of cart pole agent
# use same function of iterate batch and filter batch too.
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low) # return 1d array with 16 elements of zero
        res[observation] = 1.0 # make 1.0 at the observed space in the frozen lake. 
        return res
# because the observation space of lake is discrete, not like cart pole. 

In [134]:
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [136]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)

In [137]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convient to record the date with current time
LOGDIR = './tmp/cross_entroy/frozenlake/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR) # tf 2.0 has changed the modules of tensorboard
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    if iter_no%50 == 0: # prevent printing too many iter result 
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v.item(), iter_no)
        tf.summary.scalar("reward_bound", reward_b, iter_no)
        tf.summary.scalar("reward_mean", reward_m, iter_no)
    if reward_m > 0.8:
        print("Solved!")
        break
    if iter_no > 500: # this naive agent can't solve the problem. percentile selection is not comparable with the frozen lake returns.
        break

0: loss=1.383, reward_mean=0.1, reward_bound=0.0
50: loss=0.652, reward_mean=0.0, reward_bound=0.0
100: loss=0.355, reward_mean=0.0, reward_bound=0.0
150: loss=0.495, reward_mean=0.0, reward_bound=0.0
200: loss=0.263, reward_mean=0.0, reward_bound=0.0
250: loss=0.398, reward_mean=0.0, reward_bound=0.0
300: loss=0.074, reward_mean=0.0, reward_bound=0.0
350: loss=0.291, reward_mean=0.1, reward_bound=0.0
400: loss=0.204, reward_mean=0.0, reward_bound=0.0
450: loss=0.187, reward_mean=0.1, reward_bound=0.0
500: loss=0.119, reward_mean=0.1, reward_bound=0.0
